# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

In [201]:
import mujoco
import mujoco_viewer
import numpy as np
import scipy as sp

from autograder import autograder
from sim import Sim
from meshcat_mujoco import MeshCatVisualizer

franka_velocity_ctrl_sim = Sim(fname='./franka_emika_panda/scene_velocity_actuators.xml')
franka_torque_ctrl_sim = Sim(fname='./franka_emika_panda/scene_torque_actuators.xml')

vel_xml_path = './franka_emika_panda/scene_velocity_actuators.xml'
torque_xml_path = './franka_emika_panda/scene_torque_actuators.xml'


# ---------------------------------------
# from meshcat_mujoco import MeshCatVisualizer

# vel_xml_path = './franka_emika_panda/scene_velocity_actuators.xml'
# torque_xml_path = './franka_emika_panda/scene_torque_actuators.xml'

# vel_model   = mujoco.MjModel.from_xml_path(vel_xml_path)
# vel_data    = mujoco.MjData(vel_model)

# torque_model   = mujoco.MjModel.from_xml_path(torque_xml_path)
# torque_data    = mujoco.MjData(torque_model)



# viewer = MeshCatVisualizer(xml_path, model, data)

# i = 0 # <-- counter 
# for _ in range(10000): 

#     # use mujoco as normal with stepping function 
#     mujoco.mj_step(model, data)
#     # make sure you call the render function 
#     viewer.render()

# viewer.close()
# # close

# simulateCtrl(franka_velocity_ctrl_sim, velocity_ctrl)
# simulateCtrl(franka_torque_ctrl_sim, torque_ctrl, path)
# simulateCtrl(franka_torque_ctrl_sim, feedforward_torque_ctrl)



### Import mujoco and numpy

### Robot Info: Franka Emika Panda Robot
We will be importing the Franka Panda robot. Take a look at the robot tree in `franka_emika_panda/panda.xml` file. Rather than importing the robot, we will instead import a specific `scene_X.xml` file that specifies different joint actuation on the robot. Note that the `panda.xml` only specifies the robot body and joints. There are two other files that specify the type of actuation on the robot (`vel_act.xml` and `torque_act.xml`) which we will use to demonstrate the different levels of robot control. 

### Assignment Questions 
Go through this notebook and answer all the questions below. There are autograder helpers which will tell you whether you have good code or not. Please use those to ensure that you can safely move on the next problem. This will prevent any bugs from being carried over to the next problem.

## Rotation matrices and error signal
Before we begin, you need to write up a few functions that will help you compute errors on rotation matrices that you will use to control the robot. First, we will write two helper functions `hat` and `unhat` which converts vectors in $\mathbb{R}^3 \to \text{so}(3)$ and back. This will be used when you compute the distance between two rotation matrices. Specifically, the error between two rotation matrices can be defined using the axis-angle formulation when we discussed exponential coordinates. Recall that $R = e^{\hat{\omega}\theta}$ which is a rotation in some axis $\omega$ by the amount of $\theta$. We can convert $R$ into its components $\hat{\omega} \theta \in \mathbb{R}^3$ by using the matrix logarithm $\text{logm}$. We will then need to convert the resulting matrix back into $\mathbb{R}^3$ using the `unhat` operator. We will be using the scipy library to compute `unhat(sp.linalg.logm(R)) = w` where here `w` is combined with $\theta$. To compute the error between two rotation matrices, we simply multiply the two $R_1 R_2^\top$. For example, if $R_1 = R_{WD}$ and $R_2 = R_{WEE}$, then we want an $R$ such that $R_{WD} =R R_{WEE}$. Therefore, $R = R_{WD} R_{WEE}^\top$ is our relative rotation transform.


### Q1 a. Hat, Unhat 
Write a function `hat` that converts a vector $w\in\mathbb{R}^3 \to \text{so}(3)$ and another function `unhat` that goes the opposite direction.

In [202]:
def hat(w):
    # w_hat = w
    w_1 = w[0]
    w_2 = w[1]
    w_3 = w[2]
    w_hat = np.array([[0, -1*w_3, w_2], [w_3, 0, -1*w_1], [-1*w_2, w_1, 0]]) 
    return w_hat                                                  

def unhat(R):
    # w = R
    w_1 = R[2][1]
    w_2 = R[0][2]
    w_3 = R[1][0]
    w = np.array([w_1, w_2, w_3])
    return w


In [203]:
autograder.testQ1a(hat, unhat)

Passed Debug test


### Q1 b. Rotation error
Write a function that computes the error between two rotation matrices $R_1, R_2$.

In [204]:
def rot_err(R1, R2):
    R2_T = np.transpose(R2)
    err = np.matmul(R1, R2_T)
#     logm(err)
    err = sp.linalg.logm(err)
    err = unhat(err)
    # p = np.pi/2
    # err = p*err
    return err

In [205]:
autograder.testQ1b(rot_err)

Passed Debug test


## Inverse Kinematics Solver
We will now create the IK solver. We will use some helper functions to get the rotation, position, and jacobian of a site in the `.xml` file. Look through the files and find the tags for `site`. Specifically, look for the two tags `ee_site` and `target_site` within the `worldbody` tags. These will be the two reference frames on the robot with respect to the world frame. `ee_site` is attached to the robot hand and `target_site` is a reference frame in the world coordinates. Understand the functions below and answer the subsequent questions. 

In [206]:
# <site name="ee_site" pos="0 0 0.1" euler="0 0 -1.57079" size="0.05 0.05 0.05" type="sphere" rgba="0 0 0.5 0.5"/>
# <site name="target_site" pos="0.5 -0.4 0.5" euler="0 3.14 0" size="0.05 0.05 0.05" type="sphere" rgba="0.5 0 0 0.5"/>

def getSitePose(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output of the function returns a tuple of a rotation matric R and a position p 
        relative to the world frame. 
    """
    # mujoco specific function
    site_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    # data structure for obtaining the position and orientation of a site reference frame
    # note that rotation matrices are formated as a 9-dim vector and need to be reshaped
    p = data.site_xpos[site_id]
    R = data.site_xmat[site_id].reshape((3,3))
    return (R, p)

def getSiteJacobian(model, data, site_name):
    """
        This function takes in the robot model, data, and the site name of the reference frame. 
        The output is the total jacobian matrix. Note that mujoco gives jacobian 
        position and rotation matrices which need to be stacked together.
    """
    body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE, site_name)
    jacp = np.zeros((3, model.nv))
    jacr = np.zeros((3, model.nv))
    mujoco.mj_jacSite(model, data, jacp, jacr, body_id)
    return np.concatenate([jacp, jacr], axis=0)


## Q2. Write the IK step. 
Write function to compute $\Delta q$, for $q_{k+1} = q_k + \Delta q$, where $\Delta q$ is the solution to the minimum joint deviation based on the Taylor expansion of $f(q) - x_\text{des}$, and $f(q)$ is your forward kinematics (assume that this is given as x_\text{curr}).

In [207]:
def calcIKStep(jac, x_des, x_curr):
    """
        This function takes in the jacobian matrix of the x_\text{curr} = f(q) forward kinematics, 
        and the desired frame rotation matrix and position. 
        Assumes x_des, = (R_des, p_des), x_curr = (R_curr, p_curr)
        Output of this function computes $\Delta q$ using the jacobian.
    """
    # x_curr: f(q)
    # taylor expansion of f(q) - x_des === T[x_curr - x_des] === psudo-inverse(jac) * (x_des - x_curr)
    # print(x_des)
    # print(x_curr)
    
    jac_pseudo = sp.linalg.pinv(jac)

    rot_err_val = rot_err(x_des[0], x_curr[0])    
    # print("rot err")
    # print(rot_err_val)

    pos_err_val = np.subtract(x_des[1], x_curr[1])
    # print("pose err")
    # print(pos_err_val)
    
    err = np.concatenate([pos_err_val, rot_err_val])
    # print("err")
    # print(err)
    
    dq = np.matmul(jac_pseudo, err)
    return dq

def IKSolve(q, jac, x_des, x_curr):
    dq = calcIKStep(jac, x_des, x_curr)
    return q + dq


# Control!
We will now implement different control strategies to move our robot. We will use the function below to simulate the control method used. Make sure you run the cell and load the function. Note that for the following examples you need to calculate the error from the poses of the reference frames.

In [208]:
def simulateCtrl(sim, ctrl_method, xml_path, render=True, t_max=1000):
    sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        # viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer = MeshCatVisualizer(xml_path, sim.model, sim.data)
        # viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()
        
# -------------------------------
# vel_xml_path = './franka_emika_panda/scene_velocity_actuators.xml'
# torque_xml_path = './franka_emika_panda/scene_torque_actuators.xml'

# vel_model   = mujoco.MjModel.from_xml_path(vel_xml_path)
# vel_data    = mujoco.MjData(vel_model)

# torque_model   = mujoco.MjModel.from_xml_path(torque_xml_path)
# torque_data    = mujoco.MjData(torque_model)

# viewer = MeshCatVisualizer(xml_path, model, data)


## Q3 Velocity control
Create a controller that returns $\Delta q$ given the end-effector pose and the target site pose. Using the `calcIKStep` to compute $\Delta q$. Run the `simulateCtrl` using the `franka_velocity_ctrl_sim` simulation. Describe what you see. 

## Response:
I see the robot arm sink into the ground while moving slightly in the direction of the target. It reaches a point at which it is close to the target in the horizontal plane but is far below the target in the vertical plane. 

In [209]:
def velocity_ctrl(model, data):
    """
        Output will be a 7 dim vec that is applied to robot velocity control
        output will be calcIKstep(jac, x_des, x_curr)
    """
    P_ee = getSitePose(model, data, "ee_site")
    jac_ee = getSiteJacobian(model, data, "ee_site")
    P_target = getSitePose(model, data, "target_site")
    
    # jac_target = getSiteJacobian(model, data, "target_site")
    # print(P_target[0][2])
    # print(P_ee[0][2])
    # print(jac_target)
    
    dq = calcIKStep(jac_ee, P_target, P_ee)
    # print(dq)
    return dq 

Test your controller on the renderer

In [210]:
simulateCtrl(franka_velocity_ctrl_sim, velocity_ctrl, vel_xml_path)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7041/static/


## Q4 Torque control
Create a  basic PD controller that calculates torque values using $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can.Describe what you see. 

## Response:

The arm moves in the appropriate direction but sinks down and stops just below its target. It moves very quickly at first, then it slows as it gets closer to the desired position. 

In [211]:
def torque_ctrl(model, data):
    """
        Output will be a 7 dim vec tau = kp (qdes - qpos) + kd * (-qvel)
        where data.qpos = qpos and data.qvel = qvel are from the robot 
    """
    # parameters to tune:
    kp = 250
    kd = 65
    
    # data from robot
    qpos = data.qpos
    qvel = data.qvel
    
    x_ee = getSitePose(model, data, "ee_site")
    jac_ee = getSiteJacobian(model, data, "ee_site")
    x_target = getSitePose(model, data, "target_site")
    
    q_plus = IKSolve(qpos, jac_ee, x_target, x_ee)
    
    tau = kp*(q_plus - qpos) + kd * (-qvel)
    return tau

Test your controller on the renderer

In [212]:
simulateCtrl(franka_torque_ctrl_sim, torque_ctrl, torque_xml_path)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7042/static/


## Q4 Dynamic Comp. Torque control
Create a feed forward dynamic compensation controller that calculate target $q$ from the `IKSolve` given the end-effector pose and the target site pose. Tune the PD values as best as you can. Using the provided functions for obtaining the mass matrix and dynamic bias terms. Describe what you see. 

## Response:

The end-effector moves in a stright line and directly hits the target. The arm moves quickly and slows down as the end effector approaches the target. 

In [215]:
def getMassMatrix(model, data):
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    return mass_matrix

def getBiasTerms(model, data):
    return data.qfrc_bias

def feedforward_torque_ctrl(model, data):
    """
        Output will be a 7 dim vec for torque of the form 
        tau = M qddot_des + b
        where qddot_des = kp (qdes - qpos) + kd * (-qvel)
        where data.qpos = qpos and data.qvel = qvel are from the robot 
    """
    # parameters to tune:
    kp = 250
    kd = 65
    
    # data from robot
    qpos = data.qpos
    qvel = data.qvel
    M = getMassMatrix(model, data)
    b = getBiasTerms(model, data)
    
    # from torque control 
    x_ee = getSitePose(model, data, "ee_site")
    jac_ee = getSiteJacobian(model, data, "ee_site")
    x_target = getSitePose(model, data, "target_site")
    
    q_plus = IKSolve(qpos, jac_ee, x_target, x_ee)
    qddot_des = kp*(q_plus - qpos) + kd * (-qvel)
    
    # addition of mass and bias terms 
    tau = np.dot(M, qddot_des) + b
    return tau

Test your controller on the renderer

In [216]:
simulateCtrl(franka_torque_ctrl_sim, feedforward_torque_ctrl, torque_xml_path)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7044/static/
